In [1]:
# !pip install -U 'tensorflow[and-cuda]'

In [4]:
# Import Required Libraries
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3, Xception, ResNet50, DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import os
import numpy as np
import pandas as pd

# Enable mixed precision for faster training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.19.0
GPU Available: []


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")

print("Path to dataset files:", path)

c:\Users\junu\Documents\Python\Model Comparative Analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\junu\.cache\kagglehub\datasets\vipoooool\new-plant-diseases-dataset\versions\2


In [6]:
# train = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
# valid = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'
train = "C:\\Users\\junu\\.cache\\kagglehub\\datasets\\vipoooool\\new-plant-diseases-dataset\\versions\\2\\New Plant Diseases Dataset(Augmented)\\New Plant Diseases Dataset(Augmented)\\train"
valid = "C:\\Users\\junu\\.cache\\kagglehub\\datasets\\vipoooool\\new-plant-diseases-dataset\\versions\\2\\New Plant Diseases Dataset(Augmented)\\New Plant Diseases Dataset(Augmented)\\valid"

In [8]:
# IMPROVED: Larger image size for better feature extraction
image_size = (128, 128)
batch_size = 64

# Training dataset - use the entire train folder
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train,
    seed=123,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=True
)

# Validation dataset - use the entire valid folder (NO validation_split!)
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    valid,
    seed=123,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=False  # Don't shuffle validation data
)

# SAVE class_names BEFORE applying transformations
class_names = train_dataset.class_names
num_classes = len(class_names)

print(f"Number of classes: {num_classes}")
print(f"Image size: {image_size}")

# IMPROVED: Enhanced data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomFlip("vertical"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
])

# Apply augmentation and prefetching to training data
train_dataset = train_dataset.map(
    lambda x, y: (data_augmentation(x, training=True), y),
    num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

# Apply normalization and prefetching to validation data
normalization = tf.keras.layers.Rescaling(1./255)
val_dataset = val_dataset.map(
    lambda x, y: (normalization(x), y),
    num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

Found 70295 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.
Number of classes: 38
Image size: (128, 128)


In [9]:
# IMPROVED: Enhanced model architecture with deeper layers and dropout
def create_model(base_model, model_name):
    """
    Create an improved model with:
    - Deeper dense layers
    - Dropout for regularization
    - BatchNormalization for stability
    """
    # Initially freeze base model
    base_model.trainable = False
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    # IMPROVED: Deeper architecture with more neurons
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    # Output layer with float32 for numerical stability
    predictions = Dense(num_classes, activation='softmax', dtype='float32')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # IMPROVED: Higher initial learning rate
    model.compile(
        optimizer=Adam(learning_rate=0.001),  # Increased from 0.0001
        loss='categorical_crossentropy',
        metrics=['accuracy'],
        jit_compile=True
    )
    
    return model

# Function to fine-tune model
def fine_tune_model(model, base_model, num_layers_to_unfreeze=20):
    """
    Fine-tune the model by unfreezing top layers of base model
    """
    # Unfreeze the base model
    base_model.trainable = True
    
    # Freeze all layers except the last num_layers_to_unfreeze
    for layer in base_model.layers[:-num_layers_to_unfreeze]:
        layer.trainable = False
    
    # Recompile with lower learning rate for fine-tuning
    model.compile(
        optimizer=Adam(learning_rate=0.0001),  # Lower LR for fine-tuning
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [10]:
# Initialize models with proper input shapes
# Note: InceptionV3 and Xception require minimum 75x75, but 299x299 is optimal
# For consistency, we use 224x224 for all models

base_models = {
    'VGG16': VGG16(weights='imagenet', include_top=False, input_shape=image_size + (3,)),
    'VGG19': VGG19(weights='imagenet', include_top=False, input_shape=image_size + (3,)),
    'InceptionV3': InceptionV3(weights='imagenet', include_top=False, input_shape=image_size + (3,)),
    'Xception': Xception(weights='imagenet', include_top=False, input_shape=image_size + (3,)),
    'ResNet50': ResNet50(weights='imagenet', include_top=False, input_shape=image_size + (3,)),
    'DenseNet121': DenseNet121(weights='imagenet', include_top=False, input_shape=image_size + (3,))
}

models = {}
for name, base_model in base_models.items():
    models[name] = create_model(base_model, name)
    print(f"{name} model created successfully")

VGG16 model created successfully
VGG19 model created successfully
InceptionV3 model created successfully
Xception model created successfully
ResNet50 model created successfully
DenseNet121 model created successfully


In [11]:
# IMPROVED: Define callbacks for better training
def get_callbacks(model_name):
    """
    Create callbacks for training:
    - EarlyStopping: Stop when validation accuracy plateaus
    - ReduceLROnPlateau: Reduce learning rate when stuck
    - ModelCheckpoint: Save best model weights
    """
    callbacks = [
        # Stop training if val_accuracy doesn't improve for 5 epochs
        EarlyStopping(
            monitor='val_accuracy',
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        
        # Reduce learning rate if val_loss doesn't improve for 3 epochs
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-7,
            verbose=1
        ),
        
        # Save best model weights
        ModelCheckpoint(
            filepath=f'{model_name}_best.weights.h5',
            monitor='val_accuracy',
            save_best_only=True,
            save_weights_only=True,
            verbose=1
        )
    ]
    
    return callbacks

In [12]:
# IMPROVED: Two-stage training process
history = {}
initial_epochs = 10  # Initial training with frozen base
fine_tune_epochs = 10  # Fine-tuning with unfrozen layers
total_epochs = initial_epochs + fine_tune_epochs

print("="*80)
print("TRAINING STRATEGY:")
print(f"Stage 1: Initial training for {initial_epochs} epochs (base model frozen)")
print(f"Stage 2: Fine-tuning for {fine_tune_epochs} epochs (top layers unfrozen)")
print(f"Total epochs: {total_epochs}")
print("="*80)

for model_name, model in models.items():
    print(f"\n{'='*80}")
    print(f"TRAINING {model_name}")
    print(f"{'='*80}")
    
    # STAGE 1: Initial Training
    print(f"\n[STAGE 1] Initial training with frozen base model...")
    history_stage1 = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=initial_epochs,
        callbacks=get_callbacks(f"{model_name}_stage1")
    )
    
    # STAGE 2: Fine-tuning
    print(f"\n[STAGE 2] Fine-tuning with unfrozen top layers...")
    model = fine_tune_model(model, base_models[model_name], num_layers_to_unfreeze=20)
    
    history_stage2 = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=fine_tune_epochs,
        callbacks=get_callbacks(f"{model_name}_stage2")
    )
    
    # Combine histories
    history[model_name] = {
        'stage1': history_stage1,
        'stage2': history_stage2
    }
    
    # Save final model
    model.save(f"{model_name}_plant_disease_model_improved.h5")
    print(f"\n{model_name} training completed and model saved.")
    
    # Get predictions
    print(f"\nGenerating predictions for {model_name}...")
    y_pred = model.predict(val_dataset)
    
    # Extract true labels from the dataset
    y_true = []
    for image_batch, label_batch in val_dataset:
        y_true.append(label_batch)
    
    # Concatenate all batches into a single array
    y_true = tf.concat(y_true, axis=0).numpy()
    
    # Save predictions with the model name
    np.save(f"{model_name}_y_pred_improved.npy", y_pred)
    np.save(f"{model_name}_y_true_improved.npy", y_true)
    print(f'{model_name} predictions saved')
    
    # Calculate and display final accuracy
    y_pred_labels = y_pred.argmax(axis=1)
    y_true_labels = y_true.argmax(axis=1)
    final_accuracy = np.mean(y_pred_labels == y_true_labels)
    print(f"\n{model_name} FINAL VALIDATION ACCURACY: {final_accuracy*100:.2f}%")
    
    if final_accuracy >= 0.985:
        print(f"✓ {model_name} achieved >=98.5% accuracy!")
    else:
        print(f"✗ {model_name} did not reach 98.5% accuracy (got {final_accuracy*100:.2f}%)")

print("\n" + "="*80)
print("ALL MODELS TRAINING COMPLETED!")
print("="*80)

TRAINING STRATEGY:
Stage 1: Initial training for 10 epochs (base model frozen)
Stage 2: Fine-tuning for 10 epochs (top layers unfrozen)
Total epochs: 20

TRAINING VGG16

[STAGE 1] Initial training with frozen base model...
Epoch 1/10
   6/1099 ━━━━━━━━━━━━━━━━━━━━ 1:31:13 5s/step - accuracy: 0.0232 - loss: 4.5703

KeyboardInterrupt: 

In [ ]:
# ============================================================================
# FINAL TEST EVALUATION (Run this AFTER all training is complete)
# ============================================================================

print("\n" + "="*80)
print("FINAL TEST EVALUATION ON UNSEEN DATA")
print("="*80)

# Load test dataset
test = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/test'

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test,
    seed=123,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=False  # Important: don't shuffle test data
)

# Apply only normalization (NO augmentation for test!)
test_dataset = test_dataset.map(
    lambda x, y: (normalization(x), y),
    num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

# Evaluate each model on test set
test_results = []

for model_name in model_names:
    print(f"\n{'='*80}")
    print(f"Testing {model_name} on unseen test data...")
    print(f"{'='*80}")
    
    # Load the saved model
    model = tf.keras.models.load_model(f"{model_name}_plant_disease_model_improved.h5")
    
    # Evaluate on test set
    test_loss, test_accuracy = model.evaluate(test_dataset, verbose=1)
    
    # Get predictions
    y_test_pred = model.predict(test_dataset)
    
    # Extract true labels
    y_test_true = []
    for image_batch, label_batch in test_dataset:
        y_test_true.append(label_batch)
    y_test_true = tf.concat(y_test_true, axis=0).numpy()
    
    # Convert to class labels
    y_test_pred_labels = y_test_pred.argmax(axis=1)
    y_test_true_labels = y_test_true.argmax(axis=1)
    
    # Calculate metrics
    from sklearn.metrics import precision_score, recall_score, f1_score
    
    precision = precision_score(y_test_true_labels, y_test_pred_labels, average='weighted')
    recall = recall_score(y_test_true_labels, y_test_pred_labels, average='weighted')
    f1 = f1_score(y_test_true_labels, y_test_pred_labels, average='weighted')
    
    test_results.append({
        'Model': model_name,
        'Test Accuracy (%)': test_accuracy * 100,
        'Precision (%)': precision * 100,
        'Recall (%)': recall * 100,
        'F1-Score (%)': f1 * 100,
        'Test Loss': test_loss
    })
    
    print(f"\n{model_name} Test Results:")
    print(f"  Accuracy:  {test_accuracy*100:.2f}%")
    print(f"  Precision: {precision*100:.2f}%")
    print(f"  Recall:    {recall*100:.2f}%")
    print(f"  F1-Score:  {f1*100:.2f}%")
    print(f"  Loss:      {test_loss:.4f}")

# Create comparison table
test_comparison_df = pd.DataFrame(test_results)
test_comparison_df = test_comparison_df.sort_values('Test Accuracy (%)', ascending=False)

print("\n" + "="*80)
print("FINAL TEST RESULTS COMPARISON")
print("="*80)
print("\n" + test_comparison_df.to_string(index=False))

# Save test results
test_comparison_df.to_csv('test_results_comparison.csv', index=False)
print("\n✓ Test results saved to 'test_results_comparison.csv'")

print("\n" + "="*80)
print("EVALUATION COMPLETE!")
print("="*80)

In [ ]:
# Generate Model Comparison Table
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

model_names = ["VGG16", "VGG19", "InceptionV3", "Xception", "ResNet50", "DenseNet121"]
results = []

print("="*80)
print("MODEL COMPARISON RESULTS")
print("="*80)

for name in model_names:
    y_pred = np.load(f"{name}_y_pred_improved.npy")
    y_true = np.load(f"{name}_y_true_improved.npy")
    
    # Convert predictions and true labels from one-hot to class indices
    y_pred_labels = y_pred.argmax(axis=1)
    y_true_labels = y_true.argmax(axis=1)
    
    accuracy = np.mean(y_pred_labels == y_true_labels)
    
    # Calculate top-5 accuracy
    top5_pred = np.argsort(y_pred, axis=1)[:, -5:]
    top5_accuracy = np.mean([y_true_labels[i] in top5_pred[i] for i in range(len(y_true_labels))])
    
    results.append({
        'Model': name,
        'Accuracy (%)': accuracy * 100,
        'Top-5 Accuracy (%)': top5_accuracy * 100,
        'Meets Target (>98.5%)': '✓' if accuracy > 0.985 else '✗'
    })

comparison_df = pd.DataFrame(results)
comparison_df = comparison_df.sort_values('Accuracy (%)', ascending=False)

print("\n" + comparison_df.to_string(index=False))
print("\n" + "="*80)

# Count how many models achieved >98.5%
models_above_target = comparison_df[comparison_df['Meets Target (>98.5%)'] == '✓'].shape[0]
print(f"\nModels achieving >98.5% accuracy: {models_above_target}/{len(model_names)}")

# Save comparison table
comparison_df.to_csv('model_comparison_improved.csv', index=False)
print("\nComparison table saved to 'model_comparison_improved.csv'")

In [ ]:
# Generate detailed classification reports for each model
from sklearn.metrics import classification_report, precision_recall_fscore_support

print("="*80)
print("DETAILED CLASSIFICATION REPORTS")
print("="*80)

for name in model_names:
    y_pred = np.load(f"{name}_y_pred_improved.npy")
    y_true = np.load(f"{name}_y_true_improved.npy")
    
    # Convert both predictions and true labels from one-hot to class indices
    y_pred_labels = y_pred.argmax(axis=1)
    y_true_labels = y_true.argmax(axis=1)
    
    print(f"\n{'='*80}")
    print(f"{name} - Classification Report")
    print(f"{'='*80}")
    
    # Generate classification report
    report = classification_report(y_true_labels, y_pred_labels, target_names=class_names)
    print(report)
    
    # Save report to file
    with open(f"{name}_classification_report_improved.txt", 'w') as f:
        f.write(f"{name} - Classification Report\n")
        f.write("="*80 + "\n")
        f.write(report)
    
    print(f"Report saved to '{name}_classification_report_improved.txt'")

In [ ]:
# Visualize training history for best performing model
import matplotlib.pyplot as plt

# Get the best model based on accuracy
best_model_name = comparison_df.iloc[0]['Model']
print(f"Visualizing training history for best model: {best_model_name}")

if best_model_name in history:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Combine stage 1 and stage 2 histories
    stage1 = history[best_model_name]['stage1']
    stage2 = history[best_model_name]['stage2']
    
    # Plot accuracy
    axes[0].plot(stage1.history['accuracy'] + stage2.history['accuracy'], label='Training Accuracy')
    axes[0].plot(stage1.history['val_accuracy'] + stage2.history['val_accuracy'], label='Validation Accuracy')
    axes[0].axvline(x=len(stage1.history['accuracy']), color='r', linestyle='--', label='Fine-tuning starts')
    axes[0].axhline(y=0.985, color='g', linestyle='--', label='Target (98.5%)')
    axes[0].set_title(f'{best_model_name} - Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True)
    
    # Plot loss
    axes[1].plot(stage1.history['loss'] + stage2.history['loss'], label='Training Loss')
    axes[1].plot(stage1.history['val_loss'] + stage2.history['val_loss'], label='Validation Loss')
    axes[1].axvline(x=len(stage1.history['loss']), color='r', linestyle='--', label='Fine-tuning starts')
    axes[1].set_title(f'{best_model_name} - Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.savefig(f'{best_model_name}_training_history.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"Training history plot saved to '{best_model_name}_training_history.png'")

In [ ]:
# Final Summary
print("\n" + "="*80)
print("FINAL SUMMARY")
print("="*80)
print(f"\nTotal models trained: {len(model_names)}")
print(f"Models achieving >98.5% accuracy: {models_above_target}")
print(f"\nBest performing model: {comparison_df.iloc[0]['Model']}")
print(f"Best accuracy: {comparison_df.iloc[0]['Accuracy (%)']:.2f}%")
print("\nKey improvements implemented:")
print("  ✓ Increased image size to 224x224")
print("  ✓ Enhanced architecture with deeper layers and dropout")
print("  ✓ Two-stage training with fine-tuning")
print("  ✓ Learning rate scheduling")
print("  ✓ Early stopping and model checkpointing")
print("  ✓ Enhanced data augmentation")
print("\n" + "="*80)